In [1]:

import sympy as smp
import numpy as np
from sympy import *
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
from sympy.solvers.solveset import linsolve, nonlinsolve



In [2]:
t, g, m1, m2, m3, L1, L2, L3 = smp.symbols("t, g, m1, m2, m3, L1, L2, L3")
phi1, phi2, phi3 = smp.symbols("varphi1, varphi2, varphi3", cls = smp.Function)
phi1 = phi1(t)
phi2 = phi2(t)
phi3 = phi3(t)
dphi1 = smp.diff(phi1,t)
dphi2 = smp.diff(phi2,t)
dphi3 = smp.diff(phi3,t)
ddphi1 = smp.diff(dphi1,t)
ddphi2 = smp.diff(dphi2,t)
ddphi3 = smp.diff(dphi3,t)
x1 = L1 * smp.sin(phi1)
y1 = -L1 * smp.cos(phi1)
x2 = x1 + L2 * smp.sin(phi2)
y2 = y1 - L2 * smp.cos(phi2)
x3 = x2 + L3 * smp.sin(phi3)
y3 = y2 - L3 * smp.cos(phi3)
T = 1/2 * ( m1 * (smp.diff(x1,t)**2 + smp.diff(y1,t)**2) + m2 * (smp.diff(x2,t)**2 + smp.diff(y2,t)**2) + m3 * (smp.diff(x3,t)**2 + smp.diff(y3,t)**2))
V = m1 * g *y1 + m2 * g * y2 + m3 * g * y3
L = T - V 

ELE1 = (smp.diff(smp.diff(L, dphi1), t) - smp.diff(L, phi1)).simplify()
ELE2 = (smp.diff(smp.diff(L, dphi2), t) - smp.diff(L, phi2)).simplify()
ELE3 = (smp.diff(smp.diff(L, dphi3), t) - smp.diff(L, phi3)).simplify()
sols = smp.solve([ELE1, ELE2, ELE3], (ddphi1, ddphi2, ddphi3))




In [3]:
x1_f = smp.lambdify((phi1, phi2, phi3, L1, L2, L3), x1)
y1_f = smp.lambdify((phi1, phi2, phi3, L1, L2, L3), y1)
x2_f = smp.lambdify((phi1, phi2, phi3, L1, L2, L3), x2)
y2_f = smp.lambdify((phi1, phi2, phi3, L1, L2, L3), y2)
x3_f = smp.lambdify((phi1, phi2, phi3, L1, L2, L3), x3)
y3_f = smp.lambdify((phi1, phi2, phi3, L1, L2, L3), y3)


In [4]:
do1 = smp.lambdify((t, g, m1, m2, m3, L1, L2, L3, phi1, phi2, phi3, dphi1, dphi2, dphi3), sols[ddphi1])
do2 = smp.lambdify((t, g, m1, m2, m3, L1, L2, L3, phi1, phi2, phi3, dphi1, dphi2, dphi3), sols[ddphi2])
do3 = smp.lambdify((t, g, m1, m2, m3, L1, L2, L3, phi1, phi2, phi3, dphi1, dphi2, dphi3), sols[ddphi3])
dp1 = smp.lambdify(dphi1, dphi1)
dp2 = smp.lambdify(dphi2, dphi2)
dp3 = smp.lambdify(dphi3, dphi3)

In [5]:
def dSdt(S, t, g, m1, m2, m3, L1, L2, L3):
    phi1, phi2, phi3, op1, op2, op3 = S
    return [
        dp1(op1),
        dp2(op2),
        dp3(op3),
        do1(t, g, m1, m2,m3, L1, L2, L3, phi1, phi2, phi3, op1, op2, op3),
        do2(t, g, m1, m2,m3, L1, L2, L3, phi1, phi2, phi3, op1, op2, op3),
        do3(t, g, m1, m2,m3, L1, L2, L3, phi1, phi2, phi3, op1, op2, op3),
    ]

In [6]:
t = np.linspace(0, 40, 10000)
g = 9.81
m1=2
m2=1
m3 = 1
L1 = 2
L2 = 1
L3 = 1
y1 = []
y2 = [3,0.5,0.5,0,0,0]

ans = odeint(dSdt, y0=y2, t=t, args=(g,m1,m2, m3, L1,L2, L3 ))

phi1 = ans.T[0]
phi2 = ans.T[1]
phi3 = ans.T[2]

def get_pos(phi1, phi2, phi3, L1, L2, L3):
    i = len(x1_f(phi1, phi2, phi3, L1, L2, L3))
    return (x1_f(phi1, phi2, phi3, L1, L2, L3),
            y1_f(phi1, phi2, phi3, L1, L2, L3),
            x2_f(phi1, phi2, phi3, L1, L2, L3),
            y2_f(phi1, phi2, phi3, L1, L2, L3),
            x3_f(phi1, phi2, phi3, L1, L2, L3),
            y3_f(phi1, phi2, phi3, L1, L2, L3),  [L1 for i in range(i)], [L2 for i in range(i)], [L3 for i in range(i)])

x1, y1, x2, y2, x3, y3, L1vec, L2vec, L3vec = get_pos(ans.T[0], ans.T[1], ans.T[2], L1, L2, L3)
np.savetxt('trippelpendelpos.txt', np.array([x1, y1, x2, y2, x3, y3, L1vec, L2vec, L3vec]))
alldata = np.insert(ans, 0, t, axis = 1)
np.savetxt('trippelpendelall.txt', np.array(alldata))
